In [10]:
import json
import asyncio
import logging
from tqdm import tqdm
import pyppeteer 
import nest_asyncio

nest_asyncio.apply()

logging.basicConfig(level=logging.INFO)

# Function to scrape the title from the URL
async def scrape_page(browser, url, retries=3, delay=5):
    try:
        page = await browser.newPage()
        await page.setUserAgent(
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        )

        logging.info(f"Going to URL: {url}")
        await page.goto(url, {"waitUntil": "load", "timeout": 120000})
#class="itemBtn stretched-box yf-cex541"
        # Extracting title from the page
        data = await page.evaluate('''() => {
            const details = {};
            const titleElement = document.querySelector('h1');
            details.title = titleElement ? titleElement.innerText : 'No title found';
            details.url = window.location.href;
            details.content = document.querySelector('yf-cex541').innerText;
            return details;
        }''')

        return data

    except Exception as e:
        logging.error(f"Error scraping {url}: {e}")
        return None
    finally:
        await page.close()

# Main function to manage the scraping task
async def main():
    url = "https://finance.yahoo.com/videos/"
    browser = await pyppeteer.launch(headless=True)
    try:
        data = await scrape_page(browser, url)
        if data:
            print(json.dumps(data, indent=4))
        else:
            print("Failed to scrape data.")
    finally:
        await browser.close()

# Running the async main function
if __name__ == "__main__":
   await main()


INFO:pyppeteer.launcher:Browser listening on: ws://127.0.0.1:58569/devtools/browser/5c1ae045-647d-4158-8321-1e2abbd252cc
INFO:root:Going to URL: https://finance.yahoo.com/videos/
INFO:pyppeteer.connection:connection closed
ERROR:root:Error scraping https://finance.yahoo.com/videos/: Navigation Timeout Exceeded: 120000 ms exceeded.
INFO:pyppeteer.launcher:terminate chrome process...


PageError: Protocol Error: Connection Closed. Most likely the page has been closed.